## Azure OpenAI Playground

This example notebook will call Azure OpenAI via a Managed Identity. When using Azure Machine Learning, you can assign a system- or user-assigned managed identity to a compute instance, to authenticate against other Azure resources such as storage.

- [Assign managed identity](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-manage-compute-instance?tabs=azure-studio&view=azureml-api-2#assign-managed-identity)
- [How to configure Azure OpenAI Service with managed identities](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/managed-identity)

Assign Cognitive Services OpenAI User (only inference) or Cognitive Services OpenAI Contributor to your managed identity.



In [ ]:
%pip install --upgrade azure-ai-ml azure-identity openai

In [ ]:
API_BASE = "https://[resource-name].openai.azure.com/"
API_VERSION = "2023-05-15" # General Available

In [ ]:
from azure.identity import ManagedIdentityCredential
import os
import datetime

client_id = os.environ.get("DEFAULT_IDENTITY_CLIENT_ID")
credential = ManagedIdentityCredential(client_id=client_id)
token = None

def create_and_refresh_token():
    """Create and refresh AAD token via Managed Identity"""
    global token
    # Check if Azure token is still valid
    if not token or datetime.datetime.fromtimestamp(token.expires_on) < datetime.datetime.now():
        token = credential.get_token("https://cognitiveservices.azure.com")

### Parameters

In [ ]:
model = "gpt-35-turbo"
temperature = 0.7

max_tokens = 800
top_p = 0.95

### System Message
[System message framework and template recommendations for Large Language Models (LLMs)](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/system-message)

In [ ]:
SYSTEM_MESSAGE = "You are an AI assistant that helps people find information."

In [ ]:
messages = [
    { "role":"system", "content": SYSTEM_MESSAGE },
    { "role":"user", "content": "Hello, can you help me?" },
]

### Build Azure OpenAI request via SDK
[Quickstart: Get started using ChatGPT and GPT-4 with Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/chatgpt-quickstart?tabs=command-line&pivots=programming-language-python)

In [ ]:
import os
import openai

openai.api_type = "azure_ad"
openai.api_base = API_BASE
openai.api_version = API_VERSION

create_and_refresh_token()
openai.api_key = token.token

response = openai.ChatCompletion.create(
  engine=model,
  messages = messages,
  temperature = temperature,
  max_tokens= max_tokens,
  top_p= top_p,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

In [ ]:
response.choices[0].message.content